# Setup

In [1]:
#Setup fastai
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [2]:
#Setup
from fastbook import *
from fastai.vision.widgets import *

In [3]:
#Other Python Libs
import random
import os
import shutil

In [4]:
# Setup Dirs

NB_DIR = Path.cwd()
ASSETS = NB_DIR/'..'/'assets'
MODELS = ASSETS/'models'
DATA   = ASSETS/'classify'

# Deploy the model

In [5]:
# Import model
def label_func(x) : return x.parent.name #prevent import error: https://discuss.pytorch.org/t/error-loading-saved-model/8371

learn_inf = load_learner(MODELS/'mf_classifier.pkl')

In [6]:
#Path for data to be classified
DATA.mkdir(exist_ok=True)

In [7]:
# Header
head = widgets.HTML('<h1>Marine Fouling Classifier</h1>')
sub  = widgets.HTML('<h3>Search Images from DuckDuckGo</h3>')
# Output Widget
output = widgets.Output()
output.clear_output()

# Images
options     = ImagesCleaner()

# Search Images in DuckDuckGo
label_range = widgets.Label('Number of Images to be fetched')
txt_range   = widgets.Text()
label_query = widgets.Label('Search Query')
txt_query   = widgets.Text()
btn_search  = widgets.Button(description='Search')
btn_run     = widgets.Button(description='Classify')
ddg_img     = Image.open(ASSETS/'ddg.png').to_thumb(180,180)
ddg_logo    = widget(ddg_img)

mainBox = VBox([head,sub,label_range,txt_range,label_query,txt_query,btn_search])
header  = HBox([mainBox,ddg_logo]) 

In [8]:
# Callback Functions

def on_run(self):
    """Manages each downloaded image associated dropdown menu and acts according to the choice made"""
    #output.clear_output()
    # Delete the selected images
    for idx in options.delete(): options.fns[idx].unlink()
        
    # Update
    fnames = get_image_files(DATA)
    options.set_fns(fnames)
    
    # Classify the selected images
    classifs=[]
    for idx in options.fns: 
          pred,pred_idx,probs = learn_inf.predict(idx) #use learn_inf when fixed
          lbl_pred = widgets.Label()
          lbl_pred.value = f'Prediction: {pred}'
          lbl_prob = widgets.Label()
          lbl_prob.value = f'Probability: {probs[pred_idx]:.04f}'
          classifs.append(VBox([lbl_pred,lbl_prob]))
                    

    class_labels = carousel(classifs, width='100%')
    output.clear_output()
    with output: display(options,class_labels)
    with output: display(btn_run)
        

def searchDDG(self):
    """Fetches urls of images that corresponds to the search query in DuckDuckGo
    and download them after clearing the destination folder"""
    output.clear_output()
    if DATA.exists and DATA.is_dir:
        if len(DATA.ls()) > 0:
            for p in DATA.ls(): os.unlink(p)
                
    query   = txt_query.value
    _range  = int(txt_range.value)
    
    urls = search_images_ddg(query, max_images=_range) # returns at least 100 urls
    download_images(DATA,urls=urls[:_range])
    fnames      = get_image_files(DATA)
    options.set_fns(fnames)

    with output: display(options,btn_run)

In [9]:
# Display

btn_search.on_click(searchDDG)
btn_run.on_click(on_run)

display(header)
output

Output()

In [10]:
# Deploy

!pip install voila
!jupyter serverextension enable --sys-prefix voila 

     |████████████████████████████████| 1.7 MB 17.8 MB/s eta 0:00:01
     |████████████████████████████████| 558 kB 31.3 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 16.1 MB/s eta 0:00:01
     |████████████████████████████████| 111 kB 30.6 MB/s eta 0:00:01
     |████████████████████████████████| 397 kB 34.2 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 13.5 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 12.1 MB/s eta 0:00:01
     |████████████████████████████████| 8.8 MB 44.5 MB/s eta 0:00:01
  Attempting uninstall: nbconvert
    Found existing installation: nbconvert 5.6.1
    Uninstalling nbconvert-5.6.1:
      Successfully uninstalled nbconvert-5.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbdev 1.1.23 requires nbconvert<6, but you have nbconvert 6.4.2 which is incompatible.
Enabling: voila
- Writing